# Laplace Transform — Three Animated Views (HTML5 Inline)

This notebook **generates** and **displays** three animations related to the Laplace transform integral

\[ \mathcal{L}\{f\}(s) = \int_0^{\infty} f(t) e^{-st} \, dt, \quad s = \sigma + j\omega. \]

You’ll see:
1. **Decay View** — how the kernel \(e^{-\sigma t}\) changes with \(\sigma\).
2. **Weighting & Accumulation** — the product \(f(t)e^{-\sigma t}\) and the running integral \(\int_0^T f(\tau)e^{-\sigma \tau}d\tau\).
3. **s-plane Sweep** — for \(f(t)=e^{-t}u(t)\), \(F(s)=1/(s+1)\), visualize \(|F(\sigma+j\omega)|\) as \(\sigma\) sweeps.


## Requirements

This notebook uses only:
- `numpy`, `scipy`, `matplotlib`

For HTML5 **video** export, `matplotlib.animation.to_html5_video()` typically requires **`ffmpeg`** installed on your system.
- If `ffmpeg` is not available, the code **falls back to a JavaScript animation** with `to_jshtml()` so it still plays inline.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
from scipy import integrate

# Global plotting defaults
plt.rcParams['figure.dpi'] = 120
plt.rcParams['animation.embed_limit'] = 50_000_000  # 50MB inline limit

def show_anim(anim, fps=15, save_mp4_path=None):
    """Try to render as HTML5 video. Fall back to JSHTML if ffmpeg isn't available.
    Optionally save an MP4 if ffmpeg works.
    """
    try:
        html5 = anim.to_html5_video()  # uses ffmpeg under the hood
        if save_mp4_path is not None:
            try:
                writer = animation.FFMpegWriter(fps=fps)
                anim.save(save_mp4_path, writer=writer)
                print(f"Saved MP4 to: {save_mp4_path}")
            except Exception as e:
                print("MP4 save skipped (ffmpeg not available or failed):", e)
        return HTML(html5)
    except Exception as e:
        print("HTML5 video fallback to JSHTML (ffmpeg not found):", e)
        return HTML(anim.to_jshtml())


## 1) Decay View — $e^{-\sigma t}$

We animate the kernel \(e^{-\sigma t}\) as \(\sigma\) increases, showing how larger \(\sigma\) suppresses later times.


In [2]:
# Time axis
fs = 200
Tmax = 5.0
t = np.linspace(0, Tmax, int(fs*Tmax), endpoint=False)

# Sigma range
sigmas = np.linspace(0.0, 2.0, 40)

fig, ax = plt.subplots(figsize=(6, 3.2))
line, = ax.plot([], [], lw=2)
ax.set_xlim(0, Tmax)
ax.set_ylim(0, 1.05)
ax.set_xlabel('t')
ax.set_ylabel('e^{-σ t}')
ax.set_title('Decay of e^{-σ t} as σ increases')
legend_txt = ax.text(0.98, 0.90, '', ha='right', va='center', transform=ax.transAxes)

def init():
    line.set_data([], [])
    legend_txt.set_text('')
    return (line, legend_txt)

def animate(i):
    sigma = sigmas[i]
    y = np.exp(-sigma * t)
    line.set_data(t, y)
    legend_txt.set_text(f'σ = {sigma:.2f}')
    return (line, legend_txt)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(sigmas), interval=80, blit=True)
display(show_anim(anim, fps=12, save_mp4_path='laplace_view1_decay.mp4'))
plt.close(fig)


Saved MP4 to: laplace_view1_decay.mp4


## 2) Weighting & Accumulation — $f(t)e^{-\sigma t}$ and $\int_0^T f(\tau)e^{-\sigma \tau}d\tau$

Let \(f(t) = \sin(2\pi t)u(t)\). We show the integrand and the **running integral** as the upper limit \(T\) increases.


In [3]:
# Time axis
fs = 200
Tmax = 5.0
t = np.linspace(0, Tmax, int(fs*Tmax), endpoint=False)

sigma = 0.6
f = np.sin(2*np.pi*1.0*t)        # 1 Hz sine
kernel = np.exp(-sigma * t)
integrand = f * kernel

# Running integral by trapezoid rule
cum_int = integrate.cumulative_trapezoid(integrand, t, initial=0.0)

fig, ax = plt.subplots(figsize=(6.4, 3.6))
ax.set_xlim(0, Tmax)
ax.set_ylim(min(integrand.min()-0.2, -1.2), max(integrand.max()+0.2, 1.2))
ax.set_xlabel('t')
ax.set_title('Weighting f(t) by e^{-σ t} and accumulating the integral')
line_f, = ax.plot([], [], label='f(t)')
line_k, = ax.plot([], [], label='e^{-σ t}')
line_fk, = ax.plot([], [], label='f(t)e^{-σ t}')
ax.legend(loc='upper right')

# Secondary axis for cumulative integral
ax2 = ax.twinx()
line_cum, = ax2.plot([], [])
ax2.set_ylabel('∫_0^T f(τ)e^{-σ τ} dτ')

frames = len(t)//4
step = max(1, len(t)//frames)

def init():
    line_f.set_data([], [])
    line_k.set_data([], [])
    line_fk.set_data([], [])
    line_cum.set_data([], [])
    return (line_f, line_k, line_fk, line_cum)

def animate(i):
    idx = i*step + 1
    idx = min(idx, len(t))
    ti = t[:idx]
    line_f.set_data(t, f)
    line_k.set_data(t, kernel)
    line_fk.set_data(t, integrand)
    line_cum.set_data(ti, cum_int[:idx])
    return (line_f, line_k, line_fk, line_cum)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, interval=60, blit=False)
display(show_anim(anim, fps=15, save_mp4_path='laplace_view2_weighting.mp4'))
plt.close(fig)


Saved MP4 to: laplace_view2_weighting.mp4


## 3) s-plane Sweep — $F(s)=\frac{1}{s+1}$, ROC: $\operatorname{Re}(s) > -1$

For \(f(t)=e^{-t}u(t)\), the Laplace transform is \(F(s)=1/(s+1)\).
We sweep \(\sigma\) and plot \(|F(\sigma + j\omega)|\) vs \(\omega\) to see how shifting along the real axis affects magnitude.


In [4]:
omegas = np.linspace(0, 12*np.pi, 600)   # ω range
sigmas = np.linspace(-0.5, 2.0, 45)      # sweep σ (ROC is Re(s)>-1)

fig, ax = plt.subplots(figsize=(6.4, 3.6))
ax.set_xlim(0, np.max(omegas))
ax.set_ylim(0, 1.2)
ax.set_xlabel('ω')
ax.set_ylabel('|F(σ + jω)|')
ax.set_title('Sweep along σ+jω for F(s)=1/(s+1); ROC: Re(s) > -1')
line, = ax.plot([], [])
txt = ax.text(0.02, 0.90, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    txt.set_text('')
    return (line, txt)

def animate(i):
    sigma = sigmas[i]
    Fmag = 1.0/np.sqrt((sigma+1.0)**2 + omegas**2)
    line.set_data(omegas, Fmag)
    txt.set_text(f'σ = {sigma:.2f}  (ROC: Re(s) > -1)')
    return (line, txt)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(sigmas), interval=80, blit=True)
display(show_anim(anim, fps=12, save_mp4_path='laplace_view3_splane.mp4'))
plt.close(fig)


Saved MP4 to: laplace_view3_splane.mp4
